In [3]:
import numpy as np
from model_MLP import MLP
import torch

In [4]:
config = np.array([[ 0.,          0.0534427,   0.05344266],
                    [ 0.,         -0.10604769,  1.05260506],
                     [ 0.,          1.05260499, -0.10604768],])
print(config)

[[ 0.          0.0534427   0.05344266]
 [ 0.         -0.10604769  1.05260506]
 [ 0.          1.05260499 -0.10604768]]


In [27]:
config = np.array([[0,0,0],[0,0,1],[0,1,0]])

In [28]:
#load the model
model = MLP()
model.load_state_dict(torch.load(r'weight\MLP_model_weights.pth'))


<All keys matched successfully>

In [29]:
#convert to tensor
config = torch.tensor(config, dtype=torch.float32)
config = config.reshape(1,9)
#predict
output = model(config)
print(output*0.01)



tensor([[0.7274]], grad_fn=<MulBackward0>)


In [31]:
# predict force using gradient at the bottom layer
config.requires_grad = True
output = model(config)
output.backward()
print(-config.grad.reshape(3,3))

tensor([[ -59.2143,  317.0271,  220.0786],
        [  28.9058,  -93.5581,  -23.4878],
        [  45.1196, -226.0930, -200.3021]])


In [11]:
from datageneration import *

In [13]:
topology = create_water_topology()
simulation = setup_simulation(topology)
config = np.array([[0,0,0],[0,0,1],[0,1,0]])
potential_energy, force = calculate_potential_energy_and_forces(simulation, config)

In [26]:
print(force/10) #in angstroms

[[   0.          210.40185547  210.40185547]
 [   0.         -265.0564209    54.65456543]
 [   0.           54.65456543 -265.0564209 ]] kJ/(nm mol)


In [30]:
potential_energy*0.01

Quantity(value=0.4262383270263672, unit=kilojoule/mole)

In [33]:
import torch
import torchani
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import os

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ani_model = torchani.models.ANI1x(periodic_table_index=True).to(device).double()
# Load the weight
weight_path = r"weight\ani_model_weights.pth"
ani_model.load_state_dict(torch.load(weight_path))

C:/Users/Lenovo/anaconda3/envs/OpenMM/Lib/site-packages/torchani\resources/


<All keys matched successfully>

In [40]:
config = np.array([[[ 0.,          0.0534427,   0.05344266],
                    [ 0.,         -0.10604769,  1.05260506],
                     [ 0.,          1.05260499, -0.10604768]]])
specieset = torch.tensor([[8, 1, 1]], device=device)
config = torch.tensor(config, requires_grad=True,dtype=torch.double)

In [41]:
energy = ani_model((specieset, config)).energies

In [42]:
derivative = torch.autograd.grad(energy.sum(), config)[0]
force = -derivative

In [43]:
print(force)

tensor([[[-0.0000, -0.7204, -0.7204],
         [-0.0000, -0.0479,  0.7683],
         [-0.0000,  0.7683, -0.0479]]], dtype=torch.float64)


In [6]:
import torch
import torch.nn as nn
# import relu
import torch.nn.functional as F

# Define the simple model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.activation = nn.ReLU()
        self.fc = nn.Linear(9, 1)  # Flattening 3*3 to a single output
        
    def forward(self, x):
        x = x.view(-1, 9)  # Flatten the 3*3 tensor to 9 elements
        x = self.activation(x)
        x = self.fc(x)
        return x

# Initialize the model
model = SimpleModel()

# Example input tensor of shape (A, 3, 3)
A = 2  # Example batch size
input_tensor = torch.randn(A, 3, 3, requires_grad=True)

# Forward pass for the whole batch
outputs = model(input_tensor)

# Custom operation that forces the derivative to be 1 for each output
outputs.sum().backward()

# Output the gradients of the input tensor
print("Gradients of the input tensor:")
print(input_tensor.grad)


Gradients of the input tensor:
tensor([[[ 0.0000,  0.0000,  0.0229],
         [ 0.0000,  0.0000, -0.0516],
         [ 0.0937,  0.0000, -0.2606]],

        [[-0.0676,  0.2969,  0.0000],
         [ 0.1743,  0.1073,  0.0000],
         [ 0.0937,  0.0758, -0.2606]]])


In [7]:
# Example input tensor of shape (A, 3, 3)
A = 2  # Example batch size
input_tensor = torch.randn(A, 3, 3, requires_grad=True)

# Forward pass for the whole batch
outputs = model(input_tensor)

# Backward pass for each element in the output tensor
for i in range(A):
    model.zero_grad()  # Reset gradients for each backprop
    output = outputs[i]
    output.backward(retain_graph=True if i < A - 1 else False)
    print(f"Gradients for element {i} of output tensor:")
    print(input_tensor.grad)  # Print gradients of the input tensor
    input_tensor.grad.zero_()  # Reset gradients after use


Gradients for element 0 of output tensor:
tensor([[[ 0.0000,  0.2969,  0.0229],
         [ 0.0000,  0.0000, -0.0516],
         [ 0.0000,  0.0000, -0.2606]],

        [[-0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]]])
Gradients for element 1 of output tensor:
tensor([[[ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[-0.0676,  0.0000,  0.0229],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0758,  0.0000]]])
